In [22]:
import numpy as np

import torch
import torch.nn as nn

In [23]:
voices = np.loadtxt("F.txt", dtype=np.int8)
print(voices)

# Delete starting silence
voices = np.delete(voices, slice(8), axis=0)
print(voices)

[[ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]
 ...
 [ 0  0 51  0]
 [ 0  0 54  0]
 [ 0  0 54  0]]
[[ 0  0  0 42]
 [ 0  0  0 42]
 [ 0  0  0 42]
 ...
 [ 0  0 51  0]
 [ 0  0 54  0]
 [ 0  0 54  0]]


In [24]:
voice_one = voices[:,0]
voice_two = voices[:,1]
voice_three = voices[:,2]
voice_four = voices[:,3]

In [25]:
voice_one_unique = set(voice_one)
voice_two_unique = set(voice_two)
voice_three_unique = set(voice_three)
voice_four_unique = set(voice_four)
print("{} unique notes are found in voice one".format(len(voice_one_unique)))
print("{} unique notes are found in voice two".format(len(voice_two_unique)))
print("{} unique notes are found in voice three".format(len(voice_three_unique)))
print("{} unique notes are found in voice four".format(len(voice_four_unique)))

22 unique notes are found in voice one
27 unique notes are found in voice two
23 unique notes are found in voice three
26 unique notes are found in voice four


In [ ]:
def train_test(voice):

    train = []
    test = []
    
    # initialize how far in the past you want to look
    memory_window = 20
    
    # get a list of all the unique notes in the voice
    voice_set = set(voice)
    distribution_notes = list(voice_set)

    for i in range(len(voice) - (memory_window+1)):
        # the train data is a time-window of voices
        train.append(voice[i:i+ memory_window])
        
        # the test data is a probability one-hot encoded vector
        # create empty probability vector
        probability_vector = [0] * len(distribution_notes)
        # get the next note and find its index in the distribution
        next_note = voice[i+ memory_window + 1]
        idx = distribution_notes.index(next_note)
        # change the value at that position to 1 in the empty probability vector
        # and add it to the test set
        probability_vector[idx] = 1
        test.append(probability_vector)

    train = np.array(train)
    test = np.array(test)
    
    return train, test

In [ ]:
train, test = train_test(voice_one)

In [26]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        out, _ = self.lstm(x, (h0, c0)) 
        out = self.fc(out[:, -1, :])  
        return out

In [27]:
# Define the model
input_size = 1 
hidden_size = 32
num_layers = 1
num_classes = 1
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)